In [1]:
# app.py
import panel as pn
pn.extension('tabulator')
import param
from math import pi
import json
import pandas as pd
import numpy as np

# from comms import MQTTConnection
from manipylator.base import MovementCommand

class Q(param.Parameterized):
    q1 = param.Number(default=0, bounds=(-pi, pi))
    q2 = param.Number(default=0, bounds=(-pi, pi))
    q3 = param.Number(default=0, bounds=(-pi, pi))
    q4 = param.Number(default=0, bounds=(-pi, pi))
    q5 = param.Number(default=0, bounds=(-pi, pi))
    q6 = param.Number(default=0, bounds=(-pi, pi))
    # d = param.Dict({"q1": q1, "q2": q2}, allow_refs=True, nested_refs=True)

    def __init__(self, **params):
        # self.wdgts = [
        #     pn.Param(
        #         var
        #         # bounds=(-pi, pi),
        #         # format="{value}π", cl
        #     )
        #     for var in (self.param.q1, self.param.q2)
        # ]
        # self.gui = pn.Column(
        #     *self.wdgts[:3],
        #     *self.wdgts[3:],
        #     sizing_mode="stretch_width",
        # )
        super().__init__(**params)

    @param.depends("q1", "q2", "q3", "q4", "q5", "q6", watch=False)
    def view(self):
        # for wdgt, value in zip(self.wdgts, (self.param.q1, self.param.q2)):
        #     wdgt.value = value
        return pn.Param(self.param)

    @property
    def df(self):
        return pd.DataFrame(self.param.values(), index=[0]).drop("name", axis="columns")


pose = Q(name="pose")
target_pose = Q(name="target_pose")

pose_df = pn.pane.DataFrame(pose.df, width=400, index=False)
target_pose_df = pn.pane.DataFrame(target_pose.df, width=400, index=False)

In [2]:
pose.df

,q1,q2,q3,q4,q5,q6
0,0,0,0,0,0,0


In [3]:
# pn.Row(pose.view, target_pose.view, width=300)

In [4]:
# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, reason_code, properties):
    print(f"Connected with result code {reason_code}")
    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe("manipylator/state")
    client.subscribe("manipylator/target")
    client.subscribe("manipylator/stepper_energized")


# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    # print(msg.topic + " " + str(msg.payload))

    if msg.topic == "manipylator/state":
        d = json.loads(msg.payload)

        pose.q1 = d["q1"]
        pose.q2 = d["q2"]
        pose.q3 = d["q3"]
        pose.q4 = d["q4"]
        pose.q5 = d["q5"]
        pose.q6 = d["q6"]

        pose_df.object = pose.df

        # print(f"q1={s.pose.q1}")

    if msg.topic == "manipylator/target":
        d = json.loads(msg.payload)

        target_pose.q1 = d["q1"]
        target_pose.q2 = d["q2"]
        target_pose.q3 = d["q3"]
        target_pose.q4 = d["q4"]
        target_pose.q5 = d["q5"]
        target_pose.q6 = d["q6"]

        target_pose_df.object = target_pose.df

        # print(f"q1={s.q.q1}")
        # self.client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
        # self.client.on_connect = self.on_connect
        # self.client.on_message = self.on_message

        # # self.client.connect("localhost", 1883, 60)
        # # self.client.connect("192.168.1.8", 1883, 60)
        # # self.client.connect("192.168.1.48", 1883, 60)
        # self.client.connect(host, 1883, 60)

        # self.client.loop_start()


# import paho.mqtt.client as mqtt
from manipylator.comms import MQTTConnection

# conn = MQTTConnection("192.168.1.8")
conn = MQTTConnection("192.168.1.48")

mq = conn.client
# mq = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
# mq.connect("192.168.1.8", 1883, 60)
mq.on_connect = on_connect
mq.on_message = on_message
# mq.loop_start()

Connected with result code Success


In [5]:
df = pd.read_csv('square_qs.csv')
# df = pd.read_csv('heart_rel.csv')


selected_pose_df = pn.pane.DataFrame(
    df,
    text_align='center',
    index=False,
    justify='center',width=400,
    
)

In [6]:
# table = pn.widgets.Tabulator(df, show_index=False, selectable=False, pagination='local')
# table

In [7]:
# df.plot(kind='bar')

In [8]:
step = pn.widgets.EditableIntSlider(name='Step', fixed_start=0, fixed_end=len(df)-1, step=1, value=0)
button = pn.widgets.Button(name='Move')
# step


In [9]:


# # Callback function
# def on_button_click(event):
#     index = step.value
#     if 0 <= index < len(df):
#         output.object = df.iloc[[index]]
#     else:
#         output.object = pd.DataFrame({'Error': ['Invalid index']})

def on_button_click(event):
    index = step.value
    
    command_df = df.iloc[index]
    command = MovementCommand(**command_df.to_dict(), absolute=True)

    # conn.run_gcode_script(command.gcode_simulated)
    conn.run_gcode_script(command.gcode)



# Link the callback to the button
button.on_click(on_button_click)

Watcher(inst=Button(name='Move'), cls=<class 'panel.widgets.button.Button'>, fn=<function on_button_click at 0x77a2922a85e0>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [10]:
def on_step(value):
    selected_pose_df.object = df.iloc[[value]]
    return selected_pose_df

bound_fn = pn.bind(on_step, step.param.value)

In [11]:
pn.Column(
    pn.Row( bound_fn, '## Selected'), pn.Row(step, button), 
    # pn.Row(pose, target_pose)
    pn.Column(pn.Row(pose_df, '## Current Pose'), pn.Row(target_pose_df, '## Target Pose'))

).servable()

Column
    [0] Row
        [0] ParamFunction(function, _pane=DataFrame, defer_load=False)
        [1] Markdown(str)
    [1] Row
        [0] EditableIntSlider(end=100, fixed_end=100, fixed_start=0, name='Step', width=300)
        [1] Button(name='Move')
    [2] Column
        [0] Row
            [0] DataFrame(DataFrame, index=False, width=400)
            [1] Markdown(str)
        [1] Row
            [0] DataFrame(DataFrame, index=False, width=400)
            [1] Markdown(str)

In [12]:
# rel_df = pd.DataFrame([sub_df.sum() for sub_df in np.array_split(df.diff().dropna(), 20)])
# rel_df.head()

In [13]:
# rel_df = df - df.iloc[0]
# rel_df = pd.DataFrame([sub_df.sum() for sub_df in np.array_split(rel_df, 20)])
# rel_df.to_csv('heart_rel.csv', index=False)